In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.init as init

In [2]:
# Hyperparams
BATCH_SIZE = 16
LR = 3e-4
NUM_WORKERS = 8
EPOCHS=100
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
DEVICE = torch.device('cuda:4')

In [3]:
normMean = [0.485, 0.456, 0.406]
normStd = [0.229, 0.224, 0.225]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normTransform
    ])
testTransform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        normTransform
    ])

In [4]:
train_dataset = datasets.CIFAR10(root='../../DATASETS/classification_datasets/cifar-10-batches-py/', train=True, download=True, transform=trainTransform)
test_dataset = datasets.CIFAR10(root='../../DATASETS/classification_datasets/cifar-10-batches-py/', train=False, download=True, transform=testTransform)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Fire(nn.Module):
    def __init__(self, in_channels, num_s1, num_e1, num_e3):
        super().__init__()
        '''num_s1 = number of 1x1 convolutions in the squeeze layer
           num_e1 = number of 1x1 convolutions in the expand layer
           num_e3 = number of 3x3 convolutions in the expand layer
           in_channels = number of input channels'''

        self.in_channels = in_channels
        self.num_s1 = num_s1
        self.num_e1 = num_e1
        self.num_e3 = num_e3
        

        self.sqz = nn.Conv2d(in_channels=in_channels, out_channels=num_s1, kernel_size=1) # the squeeze part of the Fire module
        self.relu = nn.ReLU()
        self.expand_ones = nn.Conv2d(in_channels=num_s1, out_channels=num_e1, kernel_size=1) # the expand part-1x1 convolutions of the Fire module
        self.expand_threes = nn.Conv2d(in_channels=num_s1, out_channels=num_e3, kernel_size=3, padding=1) # the expand part-3x convolutions of the Fire module

    def forward(self, x):
        sqz_out = self.relu(self.sqz(x))
        expand1_out = self.relu(self.expand_ones(sqz_out))
        expand3_out = self.relu(self.expand_threes(sqz_out))
        expand_out = torch.cat((expand1_out, expand3_out), dim=1)
        out = expand_out

        return out

In [6]:
class VanillaSqueezeNet(nn.Module):
    def __init__(self, classes):
        super().__init__()
        self.classes = classes
        
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=7, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            
            Fire(96, 16, 64, 64),
            Fire(128, 16, 64, 64),
            Fire(128, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            Fire(256, 32, 128, 128),
            Fire(256, 48, 192, 192),
            Fire(384, 48, 192, 192),
            Fire(384, 64, 256, 256),
  
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            Fire(512, 64, 256, 256),
            nn.BatchNorm2d(512)
        )
        

        final_conv = nn.Conv2d(512, classes, kernel_size=1)
        
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1,1))
            
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
        

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)

        return torch.flatten(x,1)

In [8]:
x = torch.randn(8, 3, 224, 224)

x = x.to(DEVICE)
model = VanillaSqueezeNet(10).to(DEVICE)
out = model(x)
print(out.shape)

torch.Size([8, 10])


In [11]:
criterion = nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD(model.parameters(), LR,
#                             momentum=MOMENTUM,
#                             weight_decay=WEIGHT_DECAY, nesterov=True)
optimizer = torch.optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = EPOCHS)

In [13]:
def trainEpoch(device, model, loader, optimizer, criterion):
    loop = tqdm(loader)
    model.train()

    losses = []
    correct = 0
    total = 0
    for batch_id, (images, labels) in enumerate(loop):
        images, labels = images.to(device), labels.to(device)
        model = model.to(device)
       
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()

        loss = criterion(outputs, labels)
        losses.append(loss.item())


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_postfix(train_loss=loss.item(), train_accuracy=(100*correct/total))

    return np.mean(losses), 100*correct/total


def testEpoch(device, model, loader, criterion):
    
    model.eval()
    with torch.no_grad():
        losses = []
        correct = 0
        total = 0
        loop = tqdm(loader)
        for batch_id, (images, labels) in enumerate(loop):
            images, labels = images.to(device), labels.to(device)
            model = model.to(device)
        
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()

            loss = criterion(outputs, labels)
            losses.append(loss.item())

            loop.set_postfix(test_loss=loss.item(), test_accuracy=(100*correct/total))

        return np.mean(losses), 100*correct/total

In [15]:
last_best = 0
train_losses = []
train_acc = []
test_losses = []
test_acc = []
for epoch in range (EPOCHS):
    
    train_loss, train_accuracy = trainEpoch(DEVICE, model, train_loader, optimizer, criterion)
    test_loss, test_accuracy = testEpoch(DEVICE, model, test_loader, criterion)

    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    test_acc.append(test_accuracy)

    scheduler.step()

    if last_best <= test_accuracy:
        torch.save(model.state_dict(), 'vanillaSqzNet.pt')
        print("Saving new Best Model!")
        last_best = test_accuracy

    print(f"Epoch-{epoch+1} Train Loss: {train_losses[-1]:.4f} Train Accuracy: {train_acc[-1]:.4f}")
    print(f"Epoch-{epoch+1} Test Loss: {test_losses[-1]:.4f} Test Accuracy: {test_acc[-1]:.4f}")

# dict = {'train_loss': train_losses, 'train_accuracy': train_acc, 'test_loss': test_losses, 'test_accuracy': test_acc}
# df = pd.DataFrame(dict)
# df.to_csv('loss_and_acc_resnet50_baseline.csv')

100%|██████████| 625/625 [00:06<00:00, 95.09it/s, test_accuracy=59.3, test_loss=1.22]  


Saving new Best Model!
Epoch-1 Train Loss: 1.1263 Train Accuracy: 60.1620
Epoch-1 Test Loss: 1.1549 Test Accuracy: 59.3300


100%|██████████| 625/625 [00:06<00:00, 92.89it/s, test_accuracy=66.3, test_loss=0.992] 


Saving new Best Model!
Epoch-2 Train Loss: 0.9852 Train Accuracy: 65.3020
Epoch-2 Test Loss: 0.9459 Test Accuracy: 66.2700


100%|██████████| 625/625 [00:06<00:00, 91.02it/s, test_accuracy=69.9, test_loss=0.627] 


...

Epoch-98 Train Loss: 0.0022 Train Accuracy: 99.9780
Epoch-98 Test Loss: 0.7089 Test Accuracy: 86.5700


100%|██████████| 625/625 [00:06<00:00, 95.02it/s, test_accuracy=86.5, test_loss=0.798]    


Epoch-99 Train Loss: 0.0023 Train Accuracy: 99.9680
Epoch-99 Test Loss: 0.7084 Test Accuracy: 86.5300


100%|██████████| 625/625 [00:06<00:00, 96.32it/s, test_accuracy=86.7, test_loss=1.07]     

Epoch-100 Train Loss: 0.0023 Train Accuracy: 99.9820
Epoch-100 Test Loss: 0.7066 Test Accuracy: 86.7200
